## 1. 数据库风格的DataFrame合并
`merge()` 通过列索引将两个DataFrame合并起来  
参数  
- `how`：连接方式，可用值{'left', 'right', 'outer', 'inner'}，默认'inner'，各值说明如下：![title](img/marge-how.png)
- `on`：指定使用哪列作为连接键（必须是共同拥有的标签，或标签组成的list），如果没有指定，会默认使用所有共同拥有的标签的列进行连接
- `left_on` `right_on`：指定左/右对象用做连接键的列
- `left_index` `right_index`：布尔值，是否使用左/右对象的index索引用做连接键
- `sort`：布尔值，合并后的数据根据连接键进行排序，默认False
- `suffixes`：字符串值元组，用于追加到重叠标签名的末尾，以区分列来源，默认`('_x','_y')`
- `indicator`：布尔值，在合并结果中增加一列'_merge'，用来指明每行数据的来源，默认False

In [1]:
import numpy as np
import pandas as pd

In [2]:
d1 = pd.DataFrame({'key1': list('abcabcbac'), 'data1': np.random.randint(
    10, size=9), 'k': list('ZZXYYXZYX')})
d1

,key1,data1,k
0,a,3,Z
1,b,0,Z
2,c,7,X
3,a,3,Y
4,b,5,Y
5,c,8,X
6,b,2,Z
7,a,4,Y
8,c,7,X


In [3]:
d2 = pd.DataFrame({'key2': list('abd'), 'k': list('XYZ')})
d2

,key2,k
0,a,X
1,b,Y
2,d,Z


In [4]:
pd.merge(d1, d2, on='k')  # 使用共同的k列作为连接键

,key1,data1,k,key2
0,a,3,Z,d
1,b,0,Z,d
2,b,2,Z,d
3,c,7,X,a
4,c,8,X,a
5,c,7,X,a
6,a,3,Y,b
7,b,5,Y,b
8,a,4,Y,b


In [5]:
# 分别指定两个表不同名的列作为连接键，默认how='inner'交集，显示两个表连接键共有的值
pd.merge(d1, d2, left_on='key1', right_on='key2')

,key1,data1,k_x,key2,k_y
0,a,3,Z,a,X
1,a,3,Y,a,X
2,a,4,Y,a,X
3,b,0,Z,b,Y
4,b,5,Y,b,Y
5,b,2,Z,b,Y


In [6]:
# how='outer'并集，显示两个表连接键的所有的值
pd.merge(d1, d2, left_on='key1', right_on='key2', how='outer')

,key1,data1,k_x,key2,k_y
0,a,3.0,Z,a,X
1,a,3.0,Y,a,X
2,a,4.0,Y,a,X
3,b,0.0,Z,b,Y
4,b,5.0,Y,b,Y
5,b,2.0,Z,b,Y
6,c,7.0,X,NaN,NaN
7,c,8.0,X,NaN,NaN
8,c,7.0,X,NaN,NaN
9,NaN,NaN,NaN,d,Z


In [7]:
# how='left'，显示左表连接键的所有的值
# sort=True，根据连接键进行排序
# suffixes=('_L','_R')，两个表相同的列名增加后缀用以区分
# indicator=True，增加'_merge'列，用来显示所在行的数据来源
pd.merge(d1, d2, left_on='key1', right_on='key2', how='left',
         sort=True, suffixes=('_L', '_R'), indicator=True)

,key1,data1,k_L,key2,k_R,_merge
0,a,3,Z,a,X,both
1,a,3,Y,a,X,both
2,a,4,Y,a,X,both
3,b,0,Z,b,Y,both
4,b,5,Y,b,Y,both
5,b,2,Z,b,Y,both
6,c,7,X,NaN,NaN,left_only
7,c,8,X,NaN,NaN,left_only
8,c,7,X,NaN,NaN,left_only


## 2. 索引上的合并
对于连接键位于索引上时候，可以指定`left_index=True`或`right_index=True`，用以说明索引作为连接键

In [8]:
# 将d2的k列作为index，并保留k列且修改k列的值
d2.set_index('k',inplace=True)
d2

,key2
k,
X,a
Y,b
Z,d


In [9]:
# d2使用索引作为连接键和d1的k列进行连接
pd.merge(d1,d2,left_on='k',right_index=True)

,key1,data1,k,key2
0,a,3,Z,d
1,b,0,Z,d
6,b,2,Z,d
2,c,7,X,a
5,c,8,X,a
8,c,7,X,a
3,a,3,Y,b
4,b,5,Y,b
7,a,4,Y,b


对于层次化的索引，必须将多个对应的列以列表形式作为连接键来使用

In [10]:
data1 = pd.DataFrame(
    np.random.randint(1, 100, size=25).reshape((5, 5)),
    index=[list('aabbc'), list('xyyzz')],
    columns=list('ABCDE'))
data1

A   B   C   D   E
a x  80  81  73  68  96
  y   6  71  76   1   7
b y  60  46  96  53  86
  z  57  28  33  97  72
c z  11  46  57  59  29

In [11]:
data2 = pd.DataFrame(
    {'key1': list('bbaac'), 'key2': list('xzxyz'), 'value': list('XZYZX')})
data2

,key1,key2,value
0,b,x,X
1,b,z,Z
2,a,x,Y
3,a,y,Z
4,c,z,X


In [12]:
# 将右表data2中的key1和key2组成列表作为连接键，与data1的多层index进行连接
pd.merge(data1,data2,left_index=True,right_on=['key1','key2'],how='outer')

,A,B,C,D,E,key1,key2,value
2,80.0,81.0,73.0,68.0,96.0,a,x,Y
3,6.0,71.0,76.0,1.0,7.0,a,y,Z
4,60.0,46.0,96.0,53.0,86.0,b,y,NaN
1,57.0,28.0,33.0,97.0,72.0,b,z,Z
4,11.0,46.0,57.0,59.0,29.0,c,z,X
0,NaN,NaN,NaN,NaN,NaN,b,x,X


### join()方法按索引合并
- `join()`方法可以实现按索引合并两个表，但要求没有重叠的列，并且默认使用左连接。
- 一个使用列作为连接键的表也可以使用`join()`方法来合并一个使用索引作为连接键的表，需要使用`on=`来指明作为连接键的列。
- `join()`方法还可以按索引同时合并多个表，需合并的表以列表形式传入，**注意：不支持列，所有表都必须按索引合并**

In [13]:
data3 = pd.DataFrame(
    np.random.randint(1, 100, size=5).reshape((5, 1)),
    index=[list('bbaac'), list('xzxyz')],columns=['value'])
data3

value
b x     74
  z     10
a x     79
  y     41
c z     15

In [14]:
data1.join(data3) # join按索引进行合并，默认使用左连接，左表所有索引都保留

A   B   C   D   E  value
a x  80  81  73  68  96   79.0
  y   6  71  76   1   7   41.0
b y  60  46  96  53  86    NaN
  z  57  28  33  97  72   10.0
c z  11  46  57  59  29   15.0

In [15]:
data1.join(data3,how='outer') # 可以使用how来修改连接方式

A     B     C     D     E  value
a x  80.0  81.0  73.0  68.0  96.0   79.0
  y   6.0  71.0  76.0   1.0   7.0   41.0
b x   NaN   NaN   NaN   NaN   NaN   74.0
  y  60.0  46.0  96.0  53.0  86.0    NaN
  z  57.0  28.0  33.0  97.0  72.0   10.0
c z  11.0  46.0  57.0  59.0  29.0   15.0

In [16]:
# join也可以将列和索引进行合并，需指明作为连接键的列
# 注意要让有连接键列的表来调用join方法
data2.join(data1,on=['key1','key2'])

,key1,key2,value,A,B,C,D,E
0,b,x,X,NaN,NaN,NaN,NaN,NaN
1,b,z,Z,57.0,28.0,33.0,97.0,72.0
2,a,x,Y,80.0,81.0,73.0,68.0,96.0
3,a,y,Z,6.0,71.0,76.0,1.0,7.0
4,c,z,X,11.0,46.0,57.0,59.0,29.0


In [17]:
data4=data3.copy()
data4.columns=['data4_value'] # join不能有重叠的列，因此要修改列名
data4

data4_value
b x           74
  z           10
a x           79
  y           41
c z           15

In [18]:
# 使用join按索引合并多个表，以列表形式传入要合并的表
data1.join([data3,data4])

A   B   C   D   E  value  data4_value
a x  80  81  73  68  96   79.0         79.0
  y   6  71  76   1   7   41.0         41.0
b y  60  46  96  53  86    NaN          NaN
  z  57  28  33  97  72   10.0         10.0
c z  11  46  57  59  29   15.0         15.0

## 3. 轴向连接
`concat()` 在指定的轴方向上连接多个表，将表的值和索引粘合在一起
- `axis` 默认0，对列进行连接
- `keys` 列表，用以建立一个层次化索引，标识数据的来源，对于Series如果axis=1的时候使用keys，则将keys作为列名
- `join` 连接方式，默认'outer'并集，在使用'outer'时建议加上`sort=True/False`参数
- `join_axes` 指定要在其他轴上使用的索引
- `names` 列表，给层次化索引的每层索引命名
- 如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值
- `ignore_index` 布尔值，产生一组新的由数字序列组成index，适用于要合并的表没设置index值的情况

In [19]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'a', 'b'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [20]:
# 默认在行上操作对列进行连接，将值和索引连接在一起
pd.concat([s1, s2, s3])

a    0
b    1
c    2
a    3
b    4
f    5
g    6
dtype: int64

In [21]:
# keys建立一个层次化索引，标识数据的来源
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [22]:
# 在列上操作对行进行连接，声明连接方式为'inner'（交集）
pd.concat([s1, s2], axis=1, join='inner')

,0,1
a,0,3
b,1,4


In [23]:
# join_axes指定另外一个轴上的索引
# 对于Series如果axis=1的时候使用keys，则将keys作为列名
pd.concat([s1, s2], axis=1, keys=['X', 'Y'], join_axes=[['a', 'c', 'b', 'e']])

,X,Y
a,0.0,3.0
c,NaN,2.0
b,1.0,4.0
e,NaN,NaN


In [24]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=[
                   'a', 'b', 'c'], columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [25]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2),
                   index=['a', 'c'], columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [26]:
# 对于DataFrame同样的操作
pd.concat([df1, df2], sort=False, keys=['df1', 'df2'])

one  two  three  four
df1 a  0.0  1.0    NaN   NaN
    b  2.0  3.0    NaN   NaN
    c  4.0  5.0    NaN   NaN
df2 a  NaN  NaN    5.0   6.0
    c  NaN  NaN    7.0   8.0

In [27]:
# names用来给层次化索引的每层索引命名
pd.concat([df1, df2], axis=1, sort=False, keys=[
          'df1', 'df2'], names=['key1', 'key2'])

key1 df1       df2     
key2 one two three four
a      0   1   5.0  6.0
b      2   3   NaN  NaN
c      4   5   7.0  8.0

In [28]:
# 传入字典时，键值被作为keys
pd.concat({'level1': df1, 'level2': df2}, axis=1, sort=False)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [29]:
df3 = pd.DataFrame(np.random.randint(
    10, size=8).reshape(2, 4), columns=list('abcd'))
df3

,a,b,c,d
0,6,0,6,4
1,6,0,1,6


In [30]:
df4 = pd.DataFrame(np.random.randint(
    10, size=6).reshape(2, 3), columns=list('dca'))
df4

,d,c,a
0,1,9,6
1,1,1,7


In [31]:
# 没设置index值的表合并，默认情况index值直接连接
pd.concat([df3, df4], sort=False)

,a,b,c,d
0,6,0.0,6,4
1,6,0.0,1,6
0,6,NaN,9,1
1,7,NaN,1,1


In [32]:
# ignore_index=True，产生一组新的由数字序列组成index
pd.concat([df3, df4], ignore_index=True, sort=False)

,a,b,c,d
0,6,0.0,6,4
1,6,0.0,1,6
2,6,NaN,9,1
3,7,NaN,1,1


![title](img/concat.png)

## 4. 合并重叠数据
对于两个有部分索引重叠的表，如果需要合并的时候，可以使用`combine_first()`方法，在合并时将调用方法的表中缺失值用传入的表中相应位置的值来替代。  
也可以使用`combine()`方法，传入一个方法来判断重叠位置应该使用哪个表的数据。

In [33]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [41]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [55]:
# 将a中的NaN值用b中同样位置的值替代
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [100]:
# combine传入一个方法，判断是否缺失来选择从哪个表中取值
a.combine(b,lambda x,y:y if pd.isnull(x) else x )

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [89]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],'b': [np.nan, 2., np.nan, 6.],'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [90]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [80]:
# 合并时将df1表中的缺失值用df2的值来替代
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
